In [ ]:
import urllib2
from urllib2 import Request, urlopen
from metpy.io.gini import GiniFile
from siphon.catalog import TDSCatalog
from netCDF4 import num2date
import datetime
import cartopy.crs as ccrs
cat=TDSCatalog('http://thredds.ucar.edu/thredds/catalog/satellite/VIS/EAST-CONUS_1km/current/catalog.xml')
dataset_name = sorted(cat.datasets.keys())[-1]
dataset = cat.datasets[dataset_name]
remote_gini_file = urlopen(dataset.access_urls['HTTPServer'])
gini = GiniFile(remote_gini_file)
gini_ds = gini.to_dataset()
data_var = gini_ds.variables['Visible']
x = gini_ds.variables['x'][:]
y = gini_ds.variables['y'][:]
proj_var = gini_ds.variables[data_var.grid_mapping]
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=proj_var.earth_radius,
                   semiminor_axis=proj_var.earth_radius)
proj = ccrs.LambertConformal(central_longitude=proj_var.longitude_of_central_meridian,
                             central_latitude=proj_var.latitude_of_projection_origin,
                             standard_parallels=[proj_var.standard_parallel],
                             globe=globe)
time_var = gini_ds.variables['time']
timestamp = num2date(time_var[:].squeeze(), time_var.units)
print 'VIS Time:%s'%timestamp
dateZ = datetime.datetime.utcnow()
date=datetime.datetime.now()
print 'Now: %sZ' %dateZ

# !!!!!! 10 min to plot !!!!!!!!⬇⬇⬇⬇⬇⬇⬇⬇⬇

In [ ]:
# Make sure the notebook puts figures inline
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.feature as cfeat
from metpy.plots.ctables import registry
from matplotlib import patheffects
from netCDF4 import num2date
time_var = gini_ds.variables['time']
timestamp = num2date(time_var[:].squeeze(), time_var.units)
print timestamp

state_boundaries = cfeat.NaturalEarthFeature(category='cultural',
                                             name='admin_1_states_provinces_lines',
                                             scale='50m', facecolor='none')
#fig = plt.figure(figsize=(20,20))
fig=plt.figure()


#wv_norm, wv_cmap = registry.get_with_steps('WVCIMSS', 0, 1)
ax = fig.add_subplot(1, 1, 1, projection=proj)
ax.set_extent([-92,-87,43,37], ccrs.Geodetic()) #[lon_0, lon_1, lat_0, lat_1])

im = ax.imshow(data_var[:],extent=(x[0], x[-1], y[0], y[-1]), origin='upper')
#               cmap=wv_cmap, norm=wv_norm,norm=plt.Normalize(140, 255))
#im = ax.imshow(data_var[:],extent=(x[0], x[-1], y[0], y[-1]), origin='upper',
#               cmap='gist_stern',norm=plt.Normalize(190, 255))
#Upper level low 'detection':
#im = ax.imshow(data_var[:],extent=(x[0], x[-1], y[0], y[-1]), origin='upper',
#               cmap='nipy_spectral',norm=plt.Normalize(140, 150))

#ax.coastlines(resolution='50m',color='white')
ax.add_feature(state_boundaries, color='white')
#ax.add_feature(cfeat.BORDERS, linewidth='2', color='white')
'''
text = ax.text(0.99, 0.01, timestamp.strftime('%d %B %Y %H:%M:%S Z'),
               horizontalalignment='right', transform=ax.transAxes,
               color='white', fontsize='xx-large', weight='bold')

text.set_path_effects([patheffects.Stroke(linewidth=2, foreground='black'),
                       patheffects.Normal()])
'''
#fig.savefig('20x20.png', bbox_inches='tight',dpi=200)